In [0]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, DoubleType, BooleanType, DateType
     

In [0]:
configs = {"fs.azure.account.auth.type": "OAuth",
"fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
"fs.azure.account.oauth2.client.id": "place_your_id",
"fs.azure.account.oauth2.client.secret": 'place_your_id',
"fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/place_your_id/oauth2/token"}


dbutils.fs.mount(
source = "abfss://customer-sales-data@customersalesdatastorag.dfs.core.windows.net", # contrainer@storageaccountname
mount_point = "/mnt/customersales",
extra_configs = configs)

True

In [0]:
%fs
ls "/mnt/customersales"

path,name,size,modificationTime
dbfs:/mnt/customersales/raw-data/,raw-data/,0,1704733258000
dbfs:/mnt/customersales/transformed-data/,transformed-data/,0,1704733280000


In [0]:
sales_data = spark.read.format("csv").load("/mnt/customersales/raw-data/superstore_sales.csv")

In [0]:
sales_data.show()

+---------------+----------+-----------------+----------------+--------------+----------------+---------------+------------+--------------------+-----+--------+---------+----+
|            _c0|       _c1|              _c2|             _c3|           _c4|             _c5|            _c6|         _c7|                 _c8|  _c9|    _c10|     _c11|_c12|
+---------------+----------+-----------------+----------------+--------------+----------------+---------------+------------+--------------------+-----+--------+---------+----+
|       order_id|order_date|    customer_name|           state|       country|      product_id|       category|sub_category|        product_name|sales|quantity|   profit|year|
|   AG-2011-2040|  1/1/2011|  Toby Braunhardt|     Constantine|       Algeria|OFF-TEN-10000025|Office Supplies|     Storage| Tenex Lockers, Blue|  408|       2|   106.14|2011|
|  IN-2011-47883|  1/1/2011|      Joseph Holt| New South Wales|     Australia| OFF-SU-10000618|Office Supplies|    Suppl

In [0]:
sales_data = spark.read.format("csv").option("header","true").load("/mnt/customersales/raw-data/superstore_sales.csv")

In [0]:
sales_data.show()

+---------------+----------+-----------------+-----------------+--------------+----------------+---------------+------------+--------------------+-----+--------+---------+----+
|       order_id|order_date|    customer_name|            state|       country|      product_id|       category|sub_category|        product_name|sales|quantity|   profit|year|
+---------------+----------+-----------------+-----------------+--------------+----------------+---------------+------------+--------------------+-----+--------+---------+----+
|   AG-2011-2040|  1/1/2011|  Toby Braunhardt|      Constantine|       Algeria|OFF-TEN-10000025|Office Supplies|     Storage| Tenex Lockers, Blue|  408|       2|   106.14|2011|
|  IN-2011-47883|  1/1/2011|      Joseph Holt|  New South Wales|     Australia| OFF-SU-10000618|Office Supplies|    Supplies|Acme Trimmer, Hig...|  120|       3|   36.036|2011|
|   HU-2011-1220|  1/1/2011|    Annie Thurman|         Budapest|       Hungary|OFF-TEN-10001585|Office Supplies|   

In [0]:
sales_data.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- order_date: string (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- state: string (nullable = true)
 |-- country: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sub_category: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- sales: string (nullable = true)
 |-- quantity: string (nullable = true)
 |-- profit: string (nullable = true)
 |-- year: string (nullable = true)



In [0]:
sales_data = sales_data.withColumn("sales",col("sales").cast(IntegerType()))\
    .withColumn("quantity",col("quantity").cast(IntegerType()))\
    .withColumn("profit",col("profit").cast(IntegerType()))

In [0]:
sales_data.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- order_date: string (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- state: string (nullable = true)
 |-- country: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sub_category: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- sales: integer (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- profit: integer (nullable = true)
 |-- year: string (nullable = true)



In [0]:
# Find the top category with the highest number of sales
most_sales = sales_data.orderBy("sales", ascending=False).select("category","sub_category","sales","year").show()

+---------------+------------+-----+----+
|       category|sub_category|sales|year|
+---------------+------------+-----+----+
|Office Supplies|     Storage|  999|2012|
|     Technology|     Copiers|  999|2011|
|Office Supplies|     Storage|  999|2011|
|Office Supplies|  Appliances|  999|2014|
|     Technology|    Machines|  999|2011|
|      Furniture|      Tables|  998|2013|
|     Technology|     Copiers|  998|2014|
|Office Supplies|  Appliances|  997|2012|
|     Technology|     Copiers|  997|2014|
|      Furniture|   Bookcases|  996|2014|
|      Furniture|      Chairs|  996|2013|
|Office Supplies|     Storage|  995|2013|
|     Technology| Accessories|  995|2014|
|      Furniture|      Tables|  994|2012|
|Office Supplies|  Appliances|  994|2013|
|Office Supplies|  Appliances|  994|2014|
|     Technology|      Phones|  994|2011|
|      Furniture|      Chairs|  993|2011|
|     Technology|      Phones|  993|2011|
|     Technology|     Copiers|  993|2014|
+---------------+------------+----

In [0]:
# Find the countries with the highest number of sales
most_sales = sales_data.orderBy("sales", ascending=False).select("country","sales","year").show()

+--------------------+-----+----+
|             country|sales|year|
+--------------------+-----+----+
|       United States|  999|2012|
|              France|  999|2011|
|       United States|  999|2011|
|           Guatemala|  999|2014|
|       United States|  999|2011|
|               India|  998|2013|
|              Brazil|  998|2014|
|               Kenya|  997|2012|
|               China|  997|2014|
|      United Kingdom|  996|2014|
|           Guatemala|  996|2013|
|           Singapore|  995|2013|
|              Poland|  995|2014|
|      United Kingdom|  994|2012|
|              Sweden|  994|2013|
|Democratic Republ...|  994|2014|
|              France|  994|2011|
|       Cote d'Ivoire|  993|2011|
|              France|  993|2011|
|           Argentina|  993|2014|
+--------------------+-----+----+
only showing top 20 rows



In [0]:
# Find the top category with the highest number of sales
most_sales = sales_data.orderBy("sales", ascending=False).select("country","state","sales","year").show()

+--------------------+-----------------+-----+----+
|             country|            state|sales|year|
+--------------------+-----------------+-----+----+
|       United States|            Texas|  999|2012|
|              France|         Brittany|  999|2011|
|       United States|       California|  999|2011|
|           Guatemala|        Guatemala|  999|2014|
|       United States|            Texas|  999|2011|
|               India|        Karnataka|  998|2013|
|              Brazil|        S�o Paulo|  998|2014|
|               Kenya|          Nairobi|  997|2012|
|               China|        Guangdong|  997|2014|
|      United Kingdom|          England|  996|2014|
|           Guatemala|        Guatemala|  996|2013|
|           Singapore|        Singapore|  995|2013|
|              Poland|    Lesser Poland|  995|2014|
|      United Kingdom|          England|  994|2012|
|              Sweden|        Stockholm|  994|2013|
|Democratic Republ...|          Katanga|  994|2014|
|           

In [0]:
sales_data.write.option("header",'true').csv("/mnt/customersales/transformed-data/sales_data")